In [1]:
!pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.8 MB/s eta 0:00:00


In [2]:
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 58.3 MB/s eta 0:00:00


In [1]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu # Note: VideoReader and decord.cpu are not used in this image example
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

# Define build_transform and dynamic_preprocess functions
def build_transform(input_size=448):
    """Builds a transformation pipeline for image preprocessing."""
    transform = T.Compose([
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform


def dynamic_preprocess(image, image_size=448, use_thumbnail=True, max_num=12):
    """Dynamically preprocesses the image."""
    # Placeholder for dynamic preprocessing logic.
    # This example simply returns the original image in a list.
    # Replace with actual implementation if needed.
    return [image]


def load_image(image_file, input_size=448, max_num=None):
    """Loads an image and preprocesses it.
    If 'max_num' is not None, it is passed to dynamic_preprocess, otherwise a default of 12 is used."""
    image = Image.open(image_file).convert('RGB')
    transform_fn = build_transform(input_size=input_size) # Renamed to avoid conflict with T.transform

    # Determine max_num for dynamic_preprocess
    # The original logic had a slight confusion. If max_num is None, use 12.
    # If max_num is provided (not None), use that provided value.
    current_max_num = max_num if max_num is not None else 12

    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=current_max_num)

    pixel_values_list = [transform_fn(img) for img in images] # Renamed 'image' in loop to 'img'
    pixel_values_tensor = torch.stack(pixel_values_list)
    return pixel_values_tensor

# Load the model and tokenizer
path = "OpenGVLab/InternVL2_5-1B"

# Ensure CUDA is available, otherwise, this loading config might error or behave unexpectedly
if not torch.cuda.is_available():
    print("Warning: CUDA is not available. load_in_8bit=True might not work as expected or might default to CPU.")
    # If you intend to run on CPU, you might need to change model loading parameters:
    # model = AutoModel.from_pretrained(path, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, trust_remote_code=True).eval()
    # And ensure pixel_values remain on CPU.
    # For this fix, we assume CUDA is available as per the error message.

model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=True,  # This usually moves the model to GPU if available
    low_cpu_mem_usage=True,
    use_flash_attn=True, # Requires compatible GPU and toolkit
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

# Define image path
# Make sure this path is correct and the image exists.
# For example, if running in Colab, upload the image or use a sample path.
# image_path = '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg'
# Using a placeholder for testing if the image is not available
try:
    Image.open('/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg')
    image_path = '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg'
except FileNotFoundError:
    print("Warning: Image file not found. Using a dummy black image.")
    dummy_image = Image.new('RGB', (100, 100), color = 'black')
    dummy_image.save("dummy_image.jpg")
    image_path = "dummy_image.jpg"


# Load image, convert to bfloat16, and move to the model's device
pixel_values = load_image(image_path) # max_num will default to 12 as per function definition
pixel_values = pixel_values.to(torch.bfloat16).to(model.device) # Move to the same device as the model

# Define generation configuration
generation_config = dict(
    num_beams=3, # Example, common for generation
    max_new_tokens=1024,
    do_sample=False # For deterministic output with num_beams, do_sample is often False. If True, consider temperature.
)

# Single-image single-round conversation
question = '<image>\nYou are an expert Optical Character Recognition (OCR) assistant.Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image. Transcribe the text exactly as it appears on the document, character by character. You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image. Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.'

# Perform chat
try:
    response = model.chat(tokenizer, pixel_values, question, generation_config)
    print(f'User: {question}\nAssistant: {response}')
except RuntimeError as e:
    print(f"An error occurred during model.chat: {e}")
    if "out of memory" in str(e).lower():
        print("CUDA out of memory. Try reducing batch size, image resolution, or using a smaller model if possible.")
    elif "expected all tensors to be on the same device" in str(e).lower():
         print("Device mismatch error persists. Model device:", model.device, "Pixel values device:", pixel_values.device)
    else:
        # For other RuntimeErrors, you might want to inspect model.device and pixel_values.device
        print(f"Model device: {model.device}")
        if 'pixel_values' in locals():
             print(f"Pixel_values device: {pixel_values.device}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Sliding Window

FlashAttention2 is not installed.


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
You are an expert Optical Character Recognition (OCR) assistant.Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image. Transcribe the text exactly as it appears on the document, character by character. You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image. Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
Assistant: I1234568
CALIFORNIA
DRIVER LICENSE
LN CARDHOLDER
FN IMA
2570 24TH STREET
SACRAMENTO, CA 95181
DOB 08/31/1977
DD 08/31/2010
EXP 08/31/2015
CLASS C
END NONE
0831977
SEX F
HAIR BRN
EYES BRN
DD 08/30/2010
ISS 09/30/2010
